<a href="https://colab.research.google.com/github/alexlimatds/victor-doc_classification/blob/main/victor_doc_classification_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Document classification of Victor project using a BiSLTM as machine learning model

The NN architecture comes from the _VICTOR: a Dataset for Brazilian Legal Documents Classification_ paper.

Deep learning library: PyTorch

### Instaling dependencies

In [ ]:
!pip install tqdm
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186283 sha256=231934ef73da63b852a841cd198d70c900043703d746fa02569937a144cf6b28
  Stored in directory: /tmp/pip-ephem-wheel-cache-vi74ukx9/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = '/content/gdrive/My Drive/'

Mounted at /content/gdrive


### Application parameters

In [ ]:
S = 500 # sentence length
BATCH_SIZE = 64
NUM_OF_CLASSES = 6

dataset_fraction = 1.0 # fraction of train and validation datasets to be used


dataset_dir = root_dir + 'Machine Learning/Victor datasets/'
model_path = dataset_dir + 'LSTM/'
model_file = model_path + f'pytorch_model-{S}-{dataset_fraction}.pt'

### Loading and preprocessing datasets

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data
from tqdm.notebook import trange, tqdm_notebook
import numpy as np
from datetime import datetime

In [ ]:
if dataset_fraction == 1.0: # full dataset
  train_ds_file = 'train_small.csv'
  validation_ds_file = 'validation_small.csv'
else:
  train_ds_file = f'train_small.csv-croped_{dataset_fraction}.csv'
  validation_ds_file = f'validation_small.csv-croped_{dataset_fraction}.csv'

In [ ]:
import spacy

spacy_pt = spacy.load('pt')

def tokenizer(text):
  return [tok.text for tok in spacy_pt.tokenizer(text)]


In [ ]:
%%time

TEXT = data.Field(
    tokenize=tokenizer, 
    lower=True, 
    fix_length=S)
LABEL = data.Field(
    sequential=False, 
    unk_token=None)

train_data, valid_data, test_data = data.TabularDataset.splits(
    path=dataset_dir, 
    train=train_ds_file,
    validation=validation_ds_file, 
    test='test_small.csv', 
    format='csv', 
    skip_header = True, 
    fields=[(None, None), (None, None), (None, None), ('label', LABEL), (None, None), ('text', TEXT)])

CPU times: user 5min 3s, sys: 3.3 s, total: 5min 7s
Wall time: 5min 8s


In [ ]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
  (train_data, valid_data, test_data),
  sort = False, #don't sort test/validation data
  batch_sizes=(BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
  device=device)

### Model

In [ ]:
class VictorBiLSTM(nn.Module):

  def __init__(self, sentence_len, vocab_size, n_classes):
    super(VictorBiLSTM, self).__init__()
    self.hidden_dim = 200
    self.embedding_dim = 100
    self.sentence_len = sentence_len
    self.output_dim = n_classes

    self.word_embeddings = nn.Embedding(vocab_size, self.embedding_dim)
    self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, bidirectional=True)
    self.linear = nn.Linear(self.hidden_dim * sentence_len, self.output_dim)

  def forward(self, sentence):
    s_len = sentence.shape[0] #sentence length
    b_len = sentence.shape[1] #batch size
    embeds = self.word_embeddings(sentence) #embeds shape: (s_len, b_len, embedding_dim)
    lstm_out, _ = self.lstm(embeds) #lstm_out shape: (s_len, b_len, 2 * hidden_dim) => the number 2 comes because the layer is bidirectional
    sum = (
        lstm_out[:, :, :self.hidden_dim] +  # hidden states from forward layer 
        lstm_out[:, :, self.hidden_dim:])   # hidden states from backward layer 
    linear_input = torch.flatten(sum.transpose(0, 1), start_dim=1) #linear_input shape: (b_len, s_len * hidden_dim)
    x = self.linear(linear_input)
    return x

### Training functions

In [ ]:
from sklearn.metrics import f1_score

def compute_metrics(targets, predictions):
  f1_macro = f1_score(targets, np.argmax(predictions, axis=1), average='macro')
  return f1_macro

def train(model, iterator, optimizer, criterion, epoch):
  epoch_loss = 0
  model.train()
  for batch in tqdm_notebook(iterator, desc='Train', unit='batch', leave=False):
    optimizer.zero_grad()
    predictions = model(batch.text)
    loss = criterion(predictions, batch.label)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

def predict(model, iterator, set_name):
  model.eval()
  predictions = None
  targets = None
  with torch.no_grad():
    for batch in tqdm_notebook(iterator, desc=f'Predicting ({set_name})', unit='batch', leave=False):
      out = model(batch.text)
      if predictions == None:
        predictions = out
        targets = batch.label
      else:
        predictions = torch.cat([predictions, out], dim=0)
        targets = torch.cat([targets, batch.label], dim=0)
  
  return predictions.cpu().numpy(), targets.cpu().numpy()

def evaluate(model, iterator, set_name):  
  predictions, targets = predict(model, iterator, set_name)
  return compute_metrics(targets, predictions)


### Training

In [ ]:
EPOCHS = 20
learning_rate = 1e-3

model = VictorBiLSTM(S, len(TEXT.vocab), NUM_OF_CLASSES)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [ ]:
%%time
import pandas as pd
from IPython.display import display, update_display

metrics_df = pd.DataFrame(columns=['Epoch', 'Loss (train)', 'F1 macro (train)', 'F1 macro (validation)'])
metrics_display = display(metrics_df, display_id='metrics_table')

best_valid_f1 = 0.0

for epoch in range(EPOCHS):
  train_loss = train(model, train_iterator, optimizer, criterion, epoch)
  train_f1_m = evaluate(model, train_iterator, 'train set')
  valid_f1_m = evaluate(model, valid_iterator, 'validation set')
  
  #saving
  if valid_f1_m > best_valid_f1:
    best_valid_f1 = valid_f1_m
    torch.save(model.state_dict(), model_file)

  #printing
  metrics_df.loc[epoch] = [epoch + 1, train_loss, train_f1_m, valid_f1_m]
  metrics_display.update(metrics_df)

,Epoch,Loss (train),F1 macro (train),F1 macro (validation)
0,1.0,0.278695,0.847856,0.675749
1,2.0,0.131869,0.892864,0.726030
2,3.0,0.100212,0.936678,0.750952
3,4.0,0.080664,0.927089,0.745182
4,5.0,0.068060,0.950741,0.765983
5,6.0,0.056746,0.951894,0.760074
6,7.0,0.049721,0.957211,0.744346
7,8.0,0.042823,0.954190,0.755183
8,9.0,0.038568,0.955961,0.755270
9,10.0,0.035964,0.961359,0.762612


,Epoch,Loss (train),F1 macro (train),F1 macro (validation)
0,1.0,0.278695,0.847856,0.675749
1,2.0,0.131869,0.892864,0.726030
2,3.0,0.100212,0.936678,0.750952
3,4.0,0.080664,0.927089,0.745182
4,5.0,0.068060,0.950741,0.765983
5,6.0,0.056746,0.951894,0.760074
6,7.0,0.049721,0.957211,0.744346
7,8.0,0.042823,0.954190,0.755183
8,9.0,0.038568,0.955961,0.755270
9,10.0,0.035964,0.961359,0.762612


CPU times: user 1h 15min 6s, sys: 11min 9s, total: 1h 26min 15s
Wall time: 1h 27min 20s


### Evaluation

In [ ]:
def load_saved_model(file_name):
  m = VictorBiLSTM(S, len(TEXT.vocab), NUM_OF_CLASSES)
  m = m.to(device)
  m.load_state_dict(torch.load(file_name, map_location=device))
  m.eval()
  return m

# If the next line is keep commmented, it will load the saved model
#model_file = model_path + 'pytorch_model-500-1.0.pt'
model = load_saved_model(model_file)

In [ ]:
train_predictions, train_targets = predict(model, train_iterator, 'train set')
test_predictions, test_targets = predict(model, test_iterator, 'test set')

In [ ]:
from sklearn.metrics import classification_report

test_report = classification_report(
    test_targets, 
    np.argmax(test_predictions, axis=1), 
    digits=4, 
    target_names=LABEL.vocab.itos)

train_report = classification_report(
    train_targets, 
    np.argmax(train_predictions, axis=1), 
    digits=4, 
    target_names=LABEL.vocab.itos)

print(test_report)

rep_file = open(model_file + "-test_report.txt", "wt")
rep_file.write('LSTM (from the paper) evaluation report\n')
rep_file.write(f'Test {test_report}\n')
rep_file.write(f'Train {train_report}\n')
rep_file.write(f'learning rate: {learning_rate}\n')
rep_file.write(f'optimizer: {type(optimizer).__name__}\n')
rep_file.write(f'criterion: {type(criterion).__name__}\n')
rep_file.close()

                                  precision    recall  f1-score   support

                          outros     0.9592    0.9869    0.9728     85408
                   peticao_do_RE     0.8281    0.6748    0.7436      6331
agravo_em_recurso_extraordinario     0.7871    0.4579    0.5790      1841
                        sentenca     0.9262    0.6719    0.7788      1475
          acordao_de_2_instancia     0.9595    0.8681    0.9115       273
     despacho_de_admissibilidade     0.7568    0.4242    0.5437       198

                        accuracy                         0.9496     95526
                       macro avg     0.8695    0.6806    0.7549     95526
                    weighted avg     0.9463    0.9496    0.9460     95526



References:

- https://medium.com/@rohit_agrawal/using-fine-tuned-gensim-word2vec-embeddings-with-torchtext-and-pytorch-17eea2883cd
